In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul  1 11:41:02 2020

@author: user
"""


from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]
print('\n-- origin --\n',texts)
# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]
print('\n-- tokenize --\n',texts)
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]
print('\n-- corpus --\n',corpus)

testcorpus=dictionary.doc2bow(['trees'])
print('\n-- testcorpus --\n',testcorpus)

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
print('\n-- LSI space --\n',lsi)

doc = "human computer interaction"
#"human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
print('\n-- query_doc --\n',doc.lower().split())
print('\n-- vec_bow --\n',vec_bow)
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print('\n-- transformed Vector(through SVD) --\n',vec_lsi)

index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it
#index.save('/tmp/deerwester.index')
#index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')
print('\n-- making indexes (doc,similarity) --\n',type(index))
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples
print('\n-- sorted --\n')
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sims):
    print(s, documents[i])
    
